In [1]:
import calibrators
import simulators
import logging
import numpy as np

In [2]:
logging.basicConfig(filename='data/app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)

In [3]:
def print_model_status(calibrator, model):
    print('model parameters: ' + str(model.params))
    print('calibrator model parameters: ' + str(calibrator.model_params))
    print('model parameters difference: ' + str((np.asarray(model.params) - np.asarray(calibrator.model_params)).tolist()))
    print('\nsamping point: ' + str(calibrator.sample_queue[-1].inputs))
    print('sampling value: ' + str(calibrator.sample_queue[-1].output))
    print('current observe: ' + str(model.observe(calibrator.sample_queue[-1].inputs)))

# Initialize Model and Calibrator

### Initial Model Parameters

In [19]:
model_params = [0.7991353, 2.2477762]

### Initialize

In [20]:
model = simulators.SimulatorModel(list(model_params))
calib = calibrators.SlidingWindownCalibrator(model, num_resamples=1, num_samples=12)
calib.initialize()

initializing...
initialization done!
`xtol` termination condition is satisfied.
Function evaluations 18, initial cost 5.6624e-01, final cost 3.1834e-18, first-order optimality 3.81e-09.
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 7
         Function evaluations: 10
         Gradient evaluations: 16
         Hessian evaluations: 7


In [21]:
print_model_status(calib, model)

model parameters: [0.7991353, 2.2477762]
calibrator model parameters: [ 0.7991353  2.2477762]
model parameters difference: [6.854667944367066e-10, -2.258059073056984e-09]

samping point: [2.4646128029146057, 0.92945520328508757]
sampling value: 1.38777878078e-16
current observe: 1.38777878078e-16


# Tracking

### Add a Small Perturbation

In [22]:
params_change = [0, 1E-1]

In [23]:
for i in range(len(model.params)):
    model.params[i] = model_params[i] + params_change[i]

In [24]:
print_model_status(calib, model)

model parameters: [0.7991353, 2.3477762]
calibrator model parameters: [ 0.7991353  2.2477762]
model parameters difference: [6.854667944367066e-10, 0.09999999774194102]

samping point: [2.4646128029146057, 0.92945520328508757]
sampling value: 1.38777878078e-16
current observe: 0.0016038442196


### Track

In [25]:
calib.track()

tracking...
`xtol` termination condition is satisfied.
Function evaluations 15, initial cost 2.5668e+24, final cost 1.1402e-05, first-order optimality 6.60e+11.
searching minimum...
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 8
         Hessian evaluations: 4


In [26]:
print_model_status(calib, model)

model parameters: [0.7991353, 2.3477762]
calibrator model parameters: [0.8213522888945266, 2.3200935703695227]
model parameters difference: [-0.022216988894526613, 0.027682629630477518]

samping point: [2.3922953920278793, 0.87277301858328482]
sampling value: 0.000920604179198
current observe: 0.000920604179198
